<a href="https://colab.research.google.com/github/ziyadamz2/SLM/blob/Dataset/SLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import pandas as pd
import re
import json
pd.set_option('display.max_colwidth', None)

In [ ]:
kwargs = dict(split="train", streaming=False)
dataset = load_dataset("OpenLLM-France/Lucie-Training-Dataset", "Wikipedia-fr", **kwargs)

In [ ]:
dataset = dataset.to_pandas()

In [ ]:
print(dataset.columns)
print(dataset.shape)

In [ ]:
dataset=dataset.drop(columns={'source', 'id', 'language', 'date', 'author', 'url','extra','quality_signals'})

Pre-Processing

In [ ]:
def nettoyer_texte(texte: str) -> str:
    if not isinstance(texte, str):
        return ""
    # Supprimer les balises HTML
    texte = re.sub(r"<[^>]+>", "", texte)
    # Supprimer les balises Markdown (titres, listes, gras...)
    texte = re.sub(r"(^|\n)#{1,6}\s*", " ", texte)  # titres
    texte = re.sub(r"(\*{1,2}|_{1,2})(.*?)\1", r"\2", texte)  # italique/gras
    texte = re.sub(r"^\s*[-*+]\s+", " ", texte, flags=re.MULTILINE)  # listes
    texte = re.sub(r">\s*", " ", texte)  # citations
    # Remplacer \n multiples par un espace
    texte = re.sub(r"\n+", " ", texte)
    # Supprimer espaces multiples
    texte = re.sub(r"\s+", " ", texte).strip()
    return texte

dataset["text"] = dataset["text"].apply(nettoyer_texte)

In [ ]:
text_list = dataset['text'].tolist()
with open("Dataset_text/data.txt", "w", encoding="utf-8") as f:
    for line in text_list:
        f.write(line + "\n")


# Tokenizeur

## Entrainement du tokeniseur pour qu'il s'adapte au corpus du texte

In [ ]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(
    files=["Dataset_text/data.txt"],
    vocab_size=30000,
    min_frequency=2,
    special_tokens=[
        "<s>",
        "<pad>",
        "</s>",
        "<unk>",
        "<mask>",
    ],
)

tokenizer.save_model("tokenizer_bpe")

In [ ]:
dataset['text'].head()